In [19]:
# load libraries and scripts
using Plots
using FFTW
include("../scripts/tools.jl")
include("../scripts/get_grid.jl")
include("../scripts/get_params.jl")
include("../scripts/stable.jl")
include("../scripts/dynamics.jl")
include("../scripts/plotting.jl")
include("../scripts/disturbances.jl")
include("../scripts/ps_analysis.jl");

In [3]:
# load borders
border, scale_factor = import_border("../data/borders/border.json")

dx = 0.005
mesh = get_grid(border, dx); # renamed to be able to use the grid function for plotting

In [4]:
# compute or load grid parameters
contmod = get_params(mesh, scale_factor, "../data/disc/pantagruel.h5", Niter=10,
    dmax = 2*dx, patch=1900.0, bmin=52.0, tau=5.0e-6);

  5.297989 seconds (3.77 M allocations: 16.983 GiB, 4.78% gc time, 8.66% compilation time)
  3.753261 seconds (4.27 M allocations: 1.250 GiB, 10.44% gc time, 61.78% compilation time)


In [5]:
# get stable solution
compute_stable_sol!(contmod, interval = 10000, Niter = 250000, tol = 1e-7);
dm = load_discrete_model("../data/disc/pantagruel.h5", scale_factor);
th_disc = get_discrete_values(contmod.coord[contmod.isgrid,:], dm.coord, contmod.th[contmod.isgrid]);

[10000.0 7.328706000730989e-5]
[20000.0 2.2965377293626688e-5]
[30000.0 8.507224656062462e-6]
[40000.0 5.117974986923457e-6]
[50000.0 3.448676758388558e-6]
[60000.0 2.4544002302784307e-6]
[70000.0 1.823181983384714e-6]
[80000.0 1.420736850832327e-6]
[90000.0 1.1641277828378094e-6]
[100000.0 1.0006691575359383e-6]
[110000.0 8.966544252642095e-7]
[120000.0 8.305195351443473e-7]
[130000.0 7.884933591606469e-7]
[140000.0 7.617974213669498e-7]
[150000.0 7.448437097101746e-7]
[160000.0 7.340786249976361e-7]
[170000.0 7.272437856364888e-7]
[180000.0 7.229045514423049e-7]
[190000.0 7.201498030129372e-7]
[200000.0 7.184010009098074e-7]
[210000.0 7.172908202957018e-7]
[220000.0 7.165860590463424e-7]
[230000.0 7.161386647025481e-7]
[240000.0 7.158546541097266e-7]
[250000.0 7.156743615510663e-7]
 31.269048 seconds (4.31 M allocations: 81.793 GiB, 5.54% gc time, 2.25% compilation time)


In [6]:
# Second model for fourier filtering
contmod2 = get_params(mesh, scale_factor, "../data/disc/pantagruel.h5", Niter=10,
    dmax = 2*dx, patch=1900.0, bmin=52.0, tau=5.0e-7);

  3.489664 seconds (529.80 k allocations: 16.811 GiB, 4.51% gc time)
  1.177364 seconds (3.87 k allocations: 1.011 GiB, 1.65% gc time)


In [7]:
# Get frequencies
freq = zeros(contmod2.Ny, contmod2.Nx)
freqx = fftfreq(contmod2.Nx, dx)
freqy = fftfreq(contmod2.Ny, dx)
for i=1:contmod2.Ny
    for j=1:contmod2.Nx
        freq[i, j] = sqrt(freqy[i]^2 + freqx[j]^2)
    end
end

In [8]:
# Create filter mask
filter = freq .<= .2 * maximum(freq);

In [9]:
# Filter m
mfft = fft(reshape(contmod2.m, contmod2.Ny, contmod2.Nx))
mfft[.!filter] .= 0.0
mifft = real.(ifft(mfft))
mifft[.!contmod2.isgrid] .= 0.0
contmod2.m = vec(mifft);

In [10]:
# Filter d
dfft = fft(reshape(contmod2.d, contmod2.Ny, contmod2.Nx))
dfft[.!filter] .= 0.0
difft = real.(ifft(dfft))
difft[.!contmod2.isgrid] .= 0.0
contmod2.d = vec(difft);

In [11]:
# Filter bx
bxfft = fft(reshape(contmod2.bx, contmod2.Ny, contmod2.Nx))
bxfft[.!filter] .= 0.0
bxifft = real.(ifft(bxfft))
bxifft[.!contmod2.isgrid] .= 0.0
contmod2.bx = vec(bxifft);

In [12]:
# Filter by
byfft = fft(reshape(contmod2.by, contmod2.Ny, contmod2.Nx))
byfft[.!filter] .= 0.0
byifft = real.(ifft(byfft))
byifft[.!contmod2.isgrid] .= 0.0
contmod2.by = vec(byifft);

In [13]:
# Filter p
ptemp = zeros(contmod2.Ny * contmod2.Nx)
ptemp[contmod2.isgrid] = contmod2.p
pfft = fft(reshape(ptemp, contmod2.Ny, contmod2.Nx))
pfft[.!filter] .= 0.0
pifft = real.(ifft(pfft))
pifft[.!contmod2.isgrid] .= 0.0
ptemp2 = vec(pifft)[contmod2.isgrid]
ptemp2 .-= sum(ptemp2) / sum(contmod2.isgrid)
contmod2.p = ptemp2;

In [14]:
# get stable solution for filtered model
compute_stable_sol!(contmod2, interval = 10000, Niter = 250000, tol = 1e-7);

[10000.0 9.047111319127055e-5]
[20000.0 3.0717868552132543e-5]
[30000.0 1.5777396266236643e-5]
[40000.0 1.0232021236733502e-5]
[50000.0 6.8541442663772045e-6]
[60000.0 4.650279317619521e-6]
[70000.0 3.222944455094634e-6]
[80000.0 2.3038767393845916e-6]
[90000.0 1.7144683333025768e-6]
[100000.0 1.3374745000405142e-6]
[110000.0 1.0967501860026374e-6]
[120000.0 9.432006455867281e-7]
[130000.0 8.45320402342864e-7]
[140000.0 7.829513877233296e-7]
[150000.0 7.432196480472442e-7]
[160000.0 7.179125425249566e-7]
[170000.0 7.017946328513602e-7]
[180000.0 6.915298063070452e-7]
[190000.0 6.849927771668263e-7]
[200000.0 6.808298305127636e-7]
[210000.0 6.78178792012929e-7]
[220000.0 6.764905766676321e-7]
[230000.0 6.754155021937436e-7]
[240000.0 6.747308864962775e-7]
[250000.0 6.742949170135404e-7]
 30.234148 seconds (2.27 M allocations: 81.681 GiB, 5.05% gc time, 0.06% compilation time)


In [25]:
temp = zeros(size(contmod.isgrid))
temp[contmod.isgrid] = contmod.p
#println(size(contmod2.p))
hm_plot(contmod, temp, borders = [border], c = :inferno)
savefig("../figures/p_pre.svg")

temp = zeros(size(contmod.isgrid))
temp[contmod.isgrid] = contmod2.p
#println(size(contmod2.p))
hm_plot(contmod, temp, borders = [border], c = :inferno)
savefig("../figures/p_post.svg")

hm_plot(contmod, contmod.m, borders = [border], c = :inferno)
savefig("../figures/m_pre.svg")

hm_plot(contmod, contmod2.m, borders = [border], c = :inferno)
savefig("../figures/m_post.svg")

hm_plot(contmod, contmod.d, borders = [border], c = :inferno)
savefig("../figures/d_pre.svg")

hm_plot(contmod, contmod2.d, borders = [border], c = :inferno)
savefig("../figures/d_post.svg")

hm_plot(contmod, contmod.bx, borders = [border], c = :inferno)
savefig("../figures/bx_pre.svg")

hm_plot(contmod, contmod2.bx, borders = [border], c = :inferno)
savefig("../figures/bx_post.svg")

hm_plot(contmod, contmod.by, borders = [border], c = :inferno)
savefig("../figures/by_pre.svg")

hm_plot(contmod, contmod2.by, borders = [border], c = :inferno)
savefig("../figures/by_post.svg")

hm_plot(contmod, contmod.th, borders = [border], c = :inferno)
savefig("../figures/theta_pre.svg")

hm_plot(contmod, contmod2.th, borders = [border], c = :inferno)
savefig("../figures/theta_post.svg")


In [15]:
msize = 2
clim = extrema([contmod.th; contmod2.th])
fig, axs = plt.subplots(1,3, figsize=(8,2.5), gridspec_kw=py"{\"width_ratios\":[1,1,0.05]}")
axs[1].scatter(contmod.coord[contmod.isinside,2], contmod.coord[contmod.isinside,1], c=contmod.th[contmod.isinside], s=msize, cmap=CMAP, vmin=clim[1], vmax=clim[2])
axs[2].scatter(contmod2.coord[contmod2.isinside,2], contmod2.coord[contmod2.isinside,1], c=contmod2.th[contmod2.isinside], s=msize, cmap=CMAP, vmin=clim[1], vmax=clim[2])
axs[1].set_aspect("equal")
axs[2].set_aspect("equal")
axs[1].axis("off")
axs[2].axis("off")
axs[1].text(0.1, 0.9, "(a)", transform=axs[1].transAxes)
axs[2].text(0.1, 0.9, "(b)", transform=axs[2].transAxes)
fig.colorbar(axs[2].collections[1], cax=axs[3])
plt.subplots_adjust(left=0.0, right=0.9, top=1.0, bottom=0.0, wspace=0.05, hspace=0.05)
plt.savefig("../figures/thetas_fourier.pdf")

LoadError: UndefVarError: contmod not defined

In [77]:
# create an horizontal color bar

vmin = min(minimum(contmod2.th), minimum(contmod2.th))
vmax = max(maximum(contmod2.th), maximum(contmod2.th))
x = collect(0:0.01:1)
x = collect(vmin:0.01:vmax)
n = length(x)
#nlabel = 7

#dv = vmax - vmin
#dtick = round(10 * dv / nlabel) / 10
#floor(vmin/dtick):dtick:floor(vmax/dtick)
#println(x)
#, tick_direction=:out
heatmap(x, [1], reshape(x,1,n), colorbar=false, yaxis=nothing, size=(400,50), xtick=-2:0.5:1.5)
savefig("../figures/colorbar.svg")

In [ ]:
# performing a dynamical simulation on the reduced model

 contmod2.p